### Step 0. row data

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.cluster import DBSCAN

In [ ]:
def get_XY(folder_path, set_color):
    y_age = []
    all_data = []
    file_names = os.listdir(folder_path)
    for file_name in file_names:
        # Y
        age = int(file_name[2])
        y_age.append(age)
        # X
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path)
        if set_color == 'all':
            selected_data = df.iloc[:, -3:].values.T
        elif set_color == 'R':
            selected_data = df.iloc[:, -3:-2].values.T
        elif set_color == 'G':
            selected_data = df.iloc[:, -2:-1].values.T
        elif set_color == 'B':
            selected_data = df.iloc[:, -1:].values.T
        all_data.append(selected_data)
    # 轉numpy array
    data = np.array(all_data)
    reshaped_data = data.reshape((len(file_names), -1))
    y_true = np.array(y_age)
    return reshaped_data, y_true

In [ ]:
f_path = '/datasets/frames_row_data_Ruifang'
X_all, y_true = get_XY(folder_path=f_path, set_color='all')
X_red, y_true = get_XY(folder_path=f_path, set_color='R')
X_green, y_true = get_XY(folder_path=f_path, set_color='G')
X_brown, y_true = get_XY(folder_path=f_path, set_color='B')
print("X_all : ", X_all.shape, "\n", X_all[0][:10])
print("X_red : ", X_red.shape, "\n", X_red[0][:10])
print("X_green : ", X_green.shape, "\n", X_green[0][:10])
print("X_brown : ", X_brown.shape, "\n", X_brown[0][:10])
print("y_true : ", y_true.shape, "\n", y_true[0])

### Step 1-1. DTW (find reference)

In [ ]:
from dtw import dtw
def call_DTW(X, Y):
    l2_norm = lambda X, Y: (X - Y) ** 2
    dist, cost_matrix, acc_cost_matrix, path = dtw(X, Y, dist=l2_norm)
    return dist

In [ ]:
def DTWtoReference(color):
    if color == 'red':
        work_list = X_red
    if color == 'green':
        work_list = X_green
    if color == 'brown':
        work_list = X_brown
    all_dist = []
    loop_cnt = 0
    for work1 in work_list:
        loop_cnt += 1
        work1_dist = 0
        for work2 in work_list:
            dist = call_DTW(work1, work2) / (10**5)
            work1_dist += dist
        all_dist.append(work1_dist)
    # reference_data_index = all_dist.index(min(all_dist))
    # print(f"reference : index[{all_dist.index(min(all_dist))}] min={min(all_dist)}")
    return all_dist

In [ ]:
red_dist = DTWtoReference(color='red')

In [ ]:
green_dist = DTWtoReference(color='green')

In [ ]:
brown_dist = DTWtoReference(color='brown')

In [ ]:
with open('red_dist.txt', 'w') as f:
    for line in red_dist:
        f.write(str(red_dist.index(line)))
        f.write('  ')
        f.write(str(line))
        f.write('\n')
Sort_red = sorted(red_dist)
print("Red : ")
for r in Sort_red:
    print(f"[{red_dist.index(r)}] ", end='')

with open('green_dist.txt', 'w') as f:
    for line in green_dist:
        f.write(str(green_dist.index(line)))
        f.write('  ')
        f.write(str(line))
        f.write('\n')
Sort_green = sorted(green_dist)
print("\nGreen : ")
for g in Sort_green:
    print(f"[{green_dist.index(g)}] ", end='')

with open('brown_dist.txt', 'w') as f:
    for line in brown_dist:
        f.write(str(brown_dist.index(line)))
        f.write('  ')
        f.write(str(line))
        f.write('\n')
Sort_brown = sorted(brown_dist)
print("\nBrown : ")
for r in Sort_brown:
    print(f"[{brown_dist.index(r)}] ", end='')

### Step 1-2. set  reference index

In [ ]:
reference_data_index = 25

In [ ]:
from PIL import Image
import os
import matplotlib.pyplot as plt

f_path = 'C:/Users/zombie/PycharmProjects/NTUE_ThesisProject_ImageProcessing/data_testimage/data_testimage_TPH_Ruifang/'
reference_path = f_path + os.listdir(f_path)[reference_data_index]
tomato_img = Image.open(reference_path)

fig = plt.gcf()
fig.set_size_inches(2,2)
plt.imshow(tomato_img, cmap='binary')
plt.show()

### Step 2. circular DTW (find bias)

In [ ]:
# 以紅色為基準
reference_X0 = X_red[reference_data_index] # index[25]

In [ ]:
import csv
def circularDTW(color):
    # 以紅色為基準
    if color == 'red':
        work_list = X_red
    if color == 'green':
        work_list = X_green
    if color == 'brown':
        work_list = X_brown
    # 循環對齊後，紀錄偏移量 bias
    bias_list = []
    for i in range(len(work_list)):
        work = work_list[i]
        min_distance = call_DTW(work, reference_X0)
        min_work = work
        bias = 0
        for shift in range(len(work)):
            work = np.append(work[1:],[work[0]]) # 將[0]移至[last]
            dist = call_DTW(work, reference_X0)
            if dist < min_distance:
                min_distance = dist
                min_work = work
                # 偏移量 bias
                bias = shift+1
        # 紀錄偏移量 bias
        bias_list.append(bias)
    return bias_list

In [ ]:
red_bias_list = circularDTW(color='red')

In [ ]:
print(red_bias_list)
with open('get_bias_from_red_idx25.txt', 'w') as f:
    for line in red_bias_list:
        f.write(str(red_bias_list.index(line)))
        f.write('  ')
        f.write(str(line))
        f.write('\n')

In [ ]:
# with open('datasets_preprocessing/dtw_find_reference/red_dist.txt', 'r') as fp:
#     red_dist = fp.readline()
# with open('datasets_preprocessing/dtw_find_reference/green_dist.txt', 'r') as fp:
#     green_dist = fp.readline()
# with open('datasets_preprocessing/dtw_find_reference/brown_dist.txt', 'r') as fp:
#     brown_dist = fp.readline()

### Step 3. align with reference (shift)

In [ ]:
# 以index[25]紅色為基準的偏移量 red_bias_list
# shift 綠色與棕色
def shift_row_data(color):
    if color == 'green':
        work_list = X_green
        reference_X = X_green[reference_data_index] # index[25]
    if color == 'brown':
        work_list = X_brown
        reference_X = X_brown[reference_data_index] # index[25]
    # 循環對齊
    with open(f'shift_{color}.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for i in range(len(work_list)):
            red_bias = red_bias_list[i]
            work = work_list[i]
            for _ in range(red_bias):
                work = np.append(work[1:],[work[0]])
            # writerow
            new_row = work - reference_X
            writer.writerow(new_row)

In [ ]:
shift_row_data(color='red')

In [ ]:
shift_row_data(color='green')

In [ ]:
shift_row_data(color='brown')